In [1]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import fiona
from pyproj import Proj, transform

In [2]:
conn = psycopg2.connect(dbname="gis", user="postgres", password="")
rapperswil_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1683921"
zurich_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682248"
switzerland_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -51701"

rapperswil_location = [47.226, 8.818]
zurich_location = [47.3763, 8.5403]

## Get the building polygons of all pubs and restaurants in Rapperswil

In [3]:
poi_polygons = gpd.read_postgis(
    """SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE point.amenity IN ('pub', 'restaurant')
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'""".format(rapperswil_polygon_query), 
    conn, geom_col='geometry')

poi_polygons.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=rapperswil_location, zoom_start=17)

folium.GeoJson(poi_polygons).add_to(m)

m

In [4]:
import matplotlib.cm as cmx
import matplotlib.colors as colors


def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b


def style_function(feature, n_colors):
    cid = feature['properties']['cid']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#red' if cid is None else "rgb{}".format(rgb(0, n_colors, cid))
    }

def init_style_function(n_colors):
    return lambda feature: style_function(feature, n_colors)

  

## Cluster the restaurants and pubs with DBSCAN

In [5]:
poi_polygons_clustered = gpd.read_postgis(
    """SELECT polygon.way AS geometry,
              ST_ClusterDBSCAN(polygon.way, eps := 50, minpoints := 3) over () AS cid FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE point.amenity IN ('pub', 'restaurant')
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'""".format(rapperswil_polygon_query), 
    conn, geom_col='geometry')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=rapperswil_location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

## Generate convex hulls around each cluster

In [6]:
query_convex_hull = """
WITH clusters AS (
SELECT polygon.way AS geometry,
              ST_ClusterDBSCAN(polygon.way, eps := 50, minpoints := 3) over () AS cid FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE point.amenity IN ('pub', 'restaurant')
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'
)
SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS convexhull
FROM clusters
WHERE cid IS NOT NULL
GROUP BY cid;          
"""

poi_polygons_clustered = gpd.read_postgis(query_convex_hull.format(rapperswil_polygon_query), conn, geom_col='convexhull')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=rapperswil_location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

## Convex hull for clustered restaurants and pubs in Zürich

In [7]:
poi_polygons_clustered = gpd.read_postgis(query_convex_hull.format(zurich_polygon_query), conn, geom_col='convexhull')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=zurich_location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

## Do the same with more tags

In [8]:
shop_tags = ['mall', 'bakery', 'beverages', 'butcher', 'chocolate', 'coffee',
'confectionery', 'deli', 'frozen_food', 'greengrocer', 'healthfood',
'ice_cream', 'pasta', 'pastry', 'seafood', 'spices', 'tea', 'department_store',
'supermarket', 'bag', 'boutique', 'clothes', 'fashion', 'jewelry', 'leather',
'shoes', 'tailor', 'watches', 'chemist', 'cosmetics', 'hairdresser',
'medical_supply', 'electrical', 'hardware', 'electronics', 'sports',
'swimming_pool', 'collector', 'games', 'music', 'books', 'gift', 'stationery',
'ticket', 'laundry', 'pet', 'tobacco', 'toys']

amenity_tags = ['pub', 'bar', 'cafe', 'restaurant', 'pharmacy', 'bank', 'fast_food',
'food_court', 'ice_cream', 'library', 'music_school', 'school',
'language_school', 'ferry_terminal', 'clinic', 'doctors', 'hospital',
'pharmacy', 'veterinary', 'dentist', 'arts_centre', 'cinema',
'community_centre', 'casino', 'fountain', 'nightclub', 'studio', 'theatre',
'dojo', 'internet_cafe', 'marketplace', 'post_opffice', 'townhall']

leisure_tags = ['adult_gaming_centre', 'amusement_arcade', 'beach_resort',
'fitness_centre', 'garden', 'ice_rink', 'sports_centre', 'water_park']

all_tags = shop_tags + amenity_tags + leisure_tags

In [9]:
fat_query_convex_hull = """
WITH clusters AS (
SELECT polygon.way AS geometry,
              ST_ClusterDBSCAN(polygon.way, eps := 30, minpoints := 3) over () AS cid FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE (point.shop = ANY(ARRAY{})
            OR point.amenity = ANY(ARRAY{})
            OR point.leisure = ANY(ARRAY{}))
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'
)
SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS convexhull
FROM clusters
WHERE cid IS NOT NULL
GROUP BY cid;          
""".format(shop_tags, amenity_tags, leisure_tags, zurich_polygon_query)

poi_polygons_clustered = gpd.read_postgis(fat_query_convex_hull, conn, geom_col='convexhull')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=zurich_location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

## Get all nodes and polygons with the all tags (in Rapperswil)

In [10]:
query = """
(SELECT way AS geometry, name AS popup FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY{tags}) OR shop = ANY(ARRAY{tags}) OR leisure = ANY(ARRAY{tags}))
           AND st_within(way, ({polygon})))
UNION
(SELECT polygon.way AS geometry, point.name AS popup FROM planet_osm_polygon AS polygon
    INNER JOIN planet_osm_point AS point
        ON st_within(point.way, polygon.way)
    WHERE (point.amenity = ANY(ARRAY{tags}) OR point.shop = ANY(ARRAY{tags}) OR point.leisure = ANY(ARRAY{tags}))
        AND st_within(point.way, ({polygon}))
        AND polygon.building = 'yes')
        
""".format(**{'polygon': rapperswil_polygon_query, 'tags': str(all_tags)}) 

poi_polygons = gpd.read_postgis(query, conn, geom_col='geometry')

poi_polygons.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=rapperswil_location, zoom_start=17)

folium.GeoJson(poi_polygons).add_to(m)

m